### Chatbot
In this note book we build a chat bot step by step using a free api from groqcloud

## 1. install the nessary pakages
we use the framework from Microsoft, AutoGen, which requires Python version >= 3.8, < 3.13, to build the workflow. Then, the Groq Python library is also
needed for using models on Groq cloud.

In [ ]:
! pip install groq
! pip install autogen-agentchat~=0.2
! pip install "autogen-agentchat[retrievechat]~=0.2"


## 2. Get the free api from [Groq cloud](https://console.groq.com/playground) 
There is a very good [blog](https://www.geeksforgeeks.org/groq-api-with-llama-3/) demostrating the process step by step 


In [ ]:
groq_api ="you grop api here"

### 3. Set docs paths

local documents or online

Accepted file formats for `docs_path`:
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf']

In [59]:
docs_path=["data\data_description.txt"]

### 3.Set the LLM agents representing user

In [60]:
import os
import autogen
import groq
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent
from autogen.coding import LocalCommandLineCodeExecutor
import tempfile
# directory to store the code files
temp_dir = tempfile.TemporaryDirectory()
# code executor config for running code in local environment but best run llm generate code in docker environment more details in autogen tutorials.code_executon
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# set the model of LLM
config1={"config_list": [{"model": "llama-3.2-90b-text-preview", "api_key": groq_api,"api_type": "groq",}]} 
config2={"config_list": [{"model": "llama-3.2-11b-vision-preview", "api_key": groq_api,"api_type": "groq",}]}
# set the admin of the assistant agents
user_proxy=autogen.UserProxyAgent(
    name="admin",
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    code_execution_config={"executor": executor},
)
# set the doc analyzer of the assistant agents
doc_proxy=RetrieveUserProxyAgent(
    name="plan",
    retrieve_config={
        "task": "qa",
        "docs_path": docs_path,
        "get_or_create": True, 
        "overwrite": True,
    },
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="TERMINATE",
    code_execution_config={"executor": executor},
)

### 4. Prompts for assitant agents

In [61]:
message_requirment ="""You are a helpful assistant. You can ananlyze the document and give requirments to the code writer based on document."""
message_code_write="""You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""
message_check_agent=""" You are a helpful assistant. You can check the output written by others and give feedback based on the code."""

### 5. Set LLM agengts

In [62]:
req_agent = AssistantAgent(
    name="req",
    system_message=message_requirment,
    llm_config=config2,
)
code_agent = AssistantAgent(
    name="code",
    system_message=message_code_write,
    llm_config=config1,
)

### 6. Build chat group

In [ ]:
# give the description of the agents so that the groupchat manager can understand the role of the agents
req_agent.description="analyze the document and give requirments to the code writer based on document."
code_agent.description="Solve tasks using coding and language skills."
# build the groupchat 
group_chat = autogen.GroupChat(
    agents=[req_agent, code_agent,doc_proxy],
    messages=[],
    send_introductions=True,
    speaker_selection_method="round_robin", # select the speaker in round robin fashion more details in autogen tutorials
)
# set the groupchat manager
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=config1)


### 7. Chat (only based on doc)

In [ ]:
question="can you predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable.train data is in .\data\train.csv and test data is in .\data\test.csv. give me a csv file with the Id and SalePrice columns in the .\data."
chat_resluts=doc_proxy.initiate_chat(
    manager,
    message=doc_proxy.message_generator,
    problem=question,
)